In [1]:
from lxml import etree
import itertools

In [67]:
def check_ancestor(element, ancestor=["td", "th", "br", "table"]):
    if type(ancestor) == str:
        ancestor = [ancestor]
    while element is not None:
        if element.tag.lower() in ancestor:
            return True
        element = element.getparent()
    return False


def get_child_div(root):
    child_div = {}
    for element in root.iter("*"):
        if element.getparent() is None:
            continue
        ancestor_tag = element.getparent().tag
        _ = ancestor_tag.lower() in ["div" + str(i) for i in range(1, 10)]
        if _ and element.text != None and element.text.strip() != "":
            if ancestor_tag not in child_div:
                child_div[ancestor_tag] = []
            child_div[ancestor_tag].append(element.tag)
    child_div = {k: list(set(v)) for k, v in child_div.items()}
    child_div = list(set(itertools.chain.from_iterable(child_div.values())))
    # drop xref
    child_div = [tag for tag in child_div if tag.lower() != "xref"]
    return child_div

In [83]:
process_dict = {
    "ac": {
        "b": ("after", "dot"),
        "8": ("after", "bar"),
    },
    "e": {
        "52": ("before", "_"),
        "7334": ("before", "_"),
        "9145": ("before", "_"),
        "54": ("before", "_"),
        "51": ("before", "^"),
        "7501": ("after", "bar"),
    },
    "su": ("before", "^"),
    "sup": ("before", "^"),
}
def process_text(element):
    if element.text is None or element.text.strip() == "":
        return ""
    text = element.text
    if element.tag.lower() in ["ac","e"]:
        # try:
        op = None
        if process_dict.get(element.tag.lower(), None):
            if process_dict[element.tag.lower()].get(element.attrib["T"], None):
                op = process_dict[element.tag.lower()][element.attrib["T"]][0]
        # op = process_dict.get(element.tag.lower(), None).get(element.attrib["T"], None)[0]
        # except Exception as e:
        #     print(element.tag.lower(), element.attrib["T"], e)
        #     print(process_dict[element.tag.lower()])
        #     print(process_dict[element.tag.lower()][element.attrib["T"]])
        #     raise e
        if op == "before":
            text = process_dict[element.tag.lower()][element.attrib["T"]][1] + text.lstrip()
        elif op == "after":
            text = text.rstrip() + process_dict[element.tag.lower()][element.attrib["T"]][1]
        else:
            pass
    elif element.tag.lower() in ["su", "sup"]:
        text = process_dict[element.tag.lower()][1] + text.lstrip()
    if element.tail is not None and element.tail.strip() != "":
        text = text + element.tail
    return text

In [89]:
with open("title-14.xml", "r") as f:
    tree = etree.parse(f)
root = tree.getroot()
child_div = get_child_div(root)
output_lines=[]
for e_root in tree.xpath(" | ".join([f".//{tag}" for tag in child_div])):
    output_line = ""
    # if e_root.text is not None:
    #     output_line += e_root.text.strip()
    for e in e_root.iter():
        if check_ancestor(e):
            continue
        if e.tag.lower() == "table":
            continue
        output_line += process_text(e)
    # if e_root.tail is not None:
    #     output_line += e_root.tail.strip()
    output_lines.append(output_line.rstrip()+"\n")

In [91]:
with open("title-14.txt","w+") as f:
    f.writelines(output_lines)

In [44]:
EX_XML=b"""<?xml version="1.0" encoding="UTF-8"?>
<P> ABCDE<E T="52">FGHIJ</E> KLMNO<SU>PQRST</SU> UVWXY <AC T="8">ZZZZZ</AC> </P>
"""
e_root = etree.fromstring(EX_XML)
# root = tree.getroot()
child_div = ["P"]
output_lines=[]
output_line = ""
# output_line += e_root.text.strip()
for e in e_root.iter("*"):
    # if check_table_ancestor(element):
        # continue
    text = process_text(e)
    line = text
    output_line += line
if e_root.tail is not None:
    output_line += e_root.tail.strip()
output_lines.append(output_line)

In [45]:
output_lines

[' ABCDE_FGHIJ KLMNO^PQRST UVWXY ZZZZZbar']